# ML with TensorFlow Extended (TFX) -- Part 2
The puprpose of this tutorial is to show how to do end-to-end ML with TFX libraries on Google Cloud Platform. This tutorial covers:
1. Data analysis and schema generation with **TF Data Validation**.
2. Data preprocessing with **TF Transform**.
3. Model training with **TF Estimator**.
4. Model evaluation with **TF Model Analysis**.

This notebook has been tested in Jupyter on the Deep Learning VM.

## 0. Setup Python and Cloud environment

Apache Beam support for Python 3 is in alpha at the moment, so install from source to get latest bug fixes in.

In [ ]:
%pip install --upgrade grpcio_tools tensorflow_data_validation

In [ ]:
%%bash
git clone https://github.com/apache/beam
cd beam/sdks/python
python3 setup.py sdist

In [2]:
%pip install -q --upgrade './beam/sdks/python/dist/apache-beam-2.13.0.dev0.tar.gz[gcp]'

Note: you may need to restart the kernel to use updated packages.


In [9]:
import apache_beam as beam
import platform
import tensorflow as tf
import tensorflow_data_validation as tfdv
import tensorflow_transform as tft
import tornado

print('tornado version: {}'.format(tornado.version))
print('Python version: {}'.format(platform.python_version()))
print('TF version: {}'.format(tf.__version__))
print('TFT version: {}'.format(tft.__version__))
print('TFDV version: {}'.format(tfdv.__version__))
print('Apache Beam version: {}'.format(beam.__version__))

tornado version: 5.1.1
Python version: 3.5.3
TF version: 1.13.1
TFT version: 0.13.0
TFDV version: 0.13.1
Apache Beam version: 2.13.0.dev


In [3]:
PROJECT = 'cloud-training-demos'    # Replace with your PROJECT
BUCKET = 'cloud-training-demos-ml'  # Replace with your BUCKET
REGION = 'us-central1'              # Choose an available region for Cloud MLE

import os

os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION

## ensure we're using python2 env
os.environ['CLOUDSDK_PYTHON'] = 'python2'

In [4]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

## ensure we predict locally with our current Python environment
gcloud config set ml_engine/local_python `which python`

Updated property [core/project].
Updated property [compute/region].
Updated property [ml_engine/local_python].


<img valign="middle" src="images/tfx.jpeg">

### UCI Adult Dataset: https://archive.ics.uci.edu/ml/datasets/adult
Predict whether income exceeds $50K/yr based on census data. Also known as "Census Income" dataset.

In [5]:
DATA_DIR='gs://cloud-samples-data/ml-engine/census/data'

In [6]:
import os

TRAIN_DATA_FILE = os.path.join(DATA_DIR, 'adult.data.csv')
EVAL_DATA_FILE = os.path.join(DATA_DIR, 'adult.test.csv')
!gsutil ls -l $TRAIN_DATA_FILE
!gsutil ls -l $EVAL_DATA_FILE

   3974304  2018-05-10T22:21:14Z  gs://cloud-samples-data/ml-engine/census/data/adult.data.csv
TOTAL: 1 objects, 3974304 bytes (3.79 MiB)
   1986465  2018-05-10T22:21:14Z  gs://cloud-samples-data/ml-engine/census/data/adult.test.csv
TOTAL: 1 objects, 1986465 bytes (1.89 MiB)


In [20]:
HEADER = ['age', 'workclass', 'fnlwgt', 'education', 'education_num',
               'marital_status', 'occupation', 'relationship', 'race', 'gender',
               'capital_gain', 'capital_loss', 'hours_per_week',
               'native_country', 'income_bracket']

TARGET_FEATURE_NAME = 'income_bracket'
TARGET_LABELS = [' <=50K', ' >50K']
WEIGHT_COLUMN_NAME = 'fnlwgt'

## 2. Data Preprocessing
For data preprocessing and transformation, we use [TensorFlow Transform](https://www.tensorflow.org/tfx/guide/tft) to perform the following:
1. Implement transformation logic in **preprocess_fn**
2. **Analyze and transform** training data.
4. **Transform** evaluation data.
5. Save transformed **data**, transform **schema**, and trasform **logic**.

### 2.1 Implement preprocess_fn 

In [7]:
def make_preprocessing_fn(raw_schema):

  def preprocessing_fn(input_features):

    processed_features = {}

    for feature in raw_schema.feature:
      feature_name = feature.name
      
      if feature_name in ['income_bracket', 'fnlwgt']:
        processed_features[feature_name] = input_features[feature_name]
        continue

      if feature.type == 1:
        # Extract vocabulary and integerize categorical features.
        processed_features[feature_name+"_integerized"] = tft.compute_and_apply_vocabulary(input_features[feature_name], vocab_filename=feature_name)
      else:
        # normalize numeric features.
        processed_features[feature_name+"_scaled"] = tft.scale_to_z_score(input_features[feature_name])

    # Bucketize age using quantiles. 
    quantiles = tft.quantiles(input_features["age"], num_buckets=5, epsilon=0.01)
    processed_features["age_bucketized"] = tft.apply_buckets(
      input_features["age"], bucket_boundaries=quantiles)

    return processed_features

  return preprocessing_fn

### 2.2 Implement the Beam pipeline

In [15]:
def run_pipeline(args):
  import tensorflow_transform as tft
  import tensorflow_transform.beam as tft_beam
  import tensorflow_data_validation as tfdv
  from tensorflow_transform.tf_metadata import dataset_metadata
  from tensorflow_transform.tf_metadata import dataset_schema
  from tensorflow_transform.tf_metadata import schema_utils
    
  pipeline_options = beam.pipeline.PipelineOptions(flags=[], **args)
    
  raw_schema_location = args['raw_schema_location']
  raw_train_data_location = args['raw_train_data_location']
  raw_eval_data_location = args['raw_eval_data_location']
  transformed_train_data_location = args['transformed_train_data_location']
  transformed_eval_data_location = args['transformed_eval_data_location']
  transform_artefact_location = args['transform_artefact_location']
  temporary_dir = args['temporary_dir']
  runner = args['runner']
    
  print ("Raw schema location: {}".format(raw_schema_location))
  print ("Raw train data location: {}".format(raw_train_data_location))
  print ("Raw evaluation data location: {}".format(raw_eval_data_location))
  print ("Transformed train data location: {}".format(transformed_train_data_location))
  print ("Transformed evaluation data location: {}".format(transformed_eval_data_location))
  print ("Transform artefact location: {}".format(transform_artefact_location))
  print ("Temporary directory: {}".format(temporary_dir))
  print ("Runner: {}".format(runner))
  print ("")

  # Load TFDV schema and create tft schema from it.
  source_raw_schema = tfdv.load_schema_text(raw_schema_location)
  raw_feature_spec = schema_utils.schema_as_feature_spec(source_raw_schema).feature_spec
  raw_metadata = dataset_metadata.DatasetMetadata(
    dataset_schema.from_feature_spec(raw_feature_spec))

  with beam.Pipeline(runner, options=pipeline_options) as pipeline:
    with tft_beam.Context(temporary_dir):
      
      converter = tft.coders.CsvCoder(column_names=HEADER, 
        schema=raw_metadata.schema)

      ###### analyze & transform trainining data ###############################

      # Read raw training csv data.
      step = 'Train'
      print ("Reading and parsing raw training data...")
      raw_train_data = (
        pipeline
          | '{} - Read Raw Data'.format(step) >> beam.io.textio.ReadFromText(raw_train_data_location)
          | '{} - Remove Empty Rows'.format(step) >> beam.Filter(lambda line: line)
          | '{} - Decode CSV Data'.format(step) >> beam.Map(converter.decode)
        )
      
      # Create a train dataset from the data and schema.
      raw_train_dataset = (raw_train_data, raw_metadata)

      # Analyze and transform raw_train_dataset to produced transformed_train_dataset and transform_fn.
      print ("Analyzing and transforming raw training data...")
      transformed_train_dataset, transform_fn = (
        raw_train_dataset 
        | '{} - Analyze & Transform'.format(step) >> tft_beam.AnalyzeAndTransformDataset(
              make_preprocessing_fn(source_raw_schema))
      )
  
      # Get data and schema separately from the transformed_train_dataset.
      transformed_train_data, transformed_metadata = transformed_train_dataset

      # write transformed train data to sink.
      print ("Writing transformed training data...")
      _ = (
        transformed_train_data 
          | '{} - Write Transformed Data'.format(step) >> beam.io.tfrecordio.WriteToTFRecord(
            file_path_prefix=transformed_train_data_location,
            file_name_suffix=".tfrecords",
            coder=tft.coders.ExampleProtoCoder(transformed_metadata.schema))
        )

      ###### transform evaluation data #########################################

      # Read raw training csv data.
      step = 'Eval'
      print ("Reading and parsing raw evaluation data...")
      raw_eval_data = (
        pipeline
          | '{} - Read Raw Data'.format(step) >> beam.io.textio.ReadFromText(raw_eval_data_location)
          | '{} - Remove Empty Rows'.format(step) >> beam.Filter(lambda line: line)
          | '{} - Decode CSV Data'.format(step) >> beam.Map(converter.decode)
        )
      
      # Create a eval dataset from the data and schema.
      raw_eval_dataset = (raw_eval_data, raw_metadata)

      # Transform eval data based on produced transform_fn.
      print ("Transforming raw evaluation data...")
      transformed_eval_dataset = (
        (raw_eval_dataset, transform_fn) 
          | '{} - Transform'.format(step) >> tft_beam.TransformDataset()
      )

      # Get data from the transformed_eval_dataset.
      transformed_eval_data, _ = transformed_eval_dataset

      # Write transformed eval data to sink.
      print ("Writing transformed evaluation data...")
      _ = (
          transformed_eval_data 
          | '{} - Write Transformed Data'.format(step) >> beam.io.tfrecordio.WriteToTFRecord(
              file_path_prefix=transformed_eval_data_location,
              file_name_suffix=".tfrecords",
              coder=tft.coders.ExampleProtoCoder(transformed_metadata.schema))
        )

      ###### write transformation metadata #######################################################

      # Write transform_fn.
      print ("Writing transform artefacts...")
      _ = (
          transform_fn 
          | 'Write Transform Artefacts' >> tft_beam.WriteTransformFn(
              transform_artefact_location)
      )

          

### 1.4 Run data tranformation pipeline

In [16]:
%pip freeze | grep tensorflow

mesh-tensorflow==0.0.5
tensorflow==1.13.1
tensorflow-data-validation==0.13.1
tensorflow-datasets==1.0.1
tensorflow-estimator==1.13.0
tensorflow-metadata==0.13.0
tensorflow-probability==0.6.0
tensorflow-serving-api==1.13.0rc1
tensorflow-transform==0.13.0
Note: you may need to restart the kernel to use updated packages.


In [39]:
%%writefile setup.py
from setuptools import setup, find_packages

setup(name='tfxdemo',
      version='1.0',
      packages=find_packages(),
      install_requires=['tensorflow-transform==0.13.0', 
                        'tensorflow-data-validation==0.13.1'],
)

Overwriting setup.py


In [40]:
#runner = 'DirectRunner'; OUTPUT_DIR = 'output'   # on-prem
runner = 'DataflowRunner'; OUTPUT_DIR = 'gs://{}/census/tfx/'.format(BUCKET)  # on GCP

RAW_SCHEMA_LOCATION = 'raw_schema.pbtxt'
TRANSFORM_ARTEFACTS_DIR = os.path.join(OUTPUT_DIR,'transform')
TRANSFORMED_DATA_DIR = os.path.join(OUTPUT_DIR,'transformed')
TEMP_DIR = os.path.join(OUTPUT_DIR, 'tmp')

args = {
    
    'runner': runner,

    'raw_schema_location': RAW_SCHEMA_LOCATION,

    'raw_train_data_location': TRAIN_DATA_FILE,
    'raw_eval_data_location': EVAL_DATA_FILE,

    'transformed_train_data_location':  os.path.join(TRANSFORMED_DATA_DIR, "train"),
    'transformed_eval_data_location':  os.path.join(TRANSFORMED_DATA_DIR, "eval"),
    'transform_artefact_location':  TRANSFORM_ARTEFACTS_DIR,
    
    'temporary_dir': TEMP_DIR,
    'project': PROJECT,
    'temp_location': TEMP_DIR,
    'staging_location': os.path.join(OUTPUT_DIR, 'staging'),
    'max_num_workers': 8,
    'save_main_session': False,
    'setup_file': './setup.py'
}

In [41]:
if tf.gfile.Exists(OUTPUT_DIR):
  print("Removing {} contents...".format(OUTPUT_DIR))
  tf.gfile.DeleteRecursively(OUTPUT_DIR)

tf.logging.set_verbosity(tf.logging.ERROR)
print("Running TF Transform pipeline...")
print("")
run_pipeline(args)
print("")
print("Pipeline is done.")

Removing gs://cloud-training-demos-ml/census/tfx/ contents...
Running TF Transform pipeline...

Raw schema location: raw_schema.pbtxt
Raw train data location: gs://cloud-samples-data/ml-engine/census/data/adult.data.csv
Raw evaluation data location: gs://cloud-samples-data/ml-engine/census/data/adult.test.csv
Transformed train data location: gs://cloud-training-demos-ml/census/tfx/transformed/train
Transformed evaluation data location: gs://cloud-training-demos-ml/census/tfx/transformed/eval
Transform artefact location: gs://cloud-training-demos-ml/census/tfx/transform
Temporary directory: gs://cloud-training-demos-ml/census/tfx/tmp
Runner: DataflowRunner

Reading and parsing raw training data...
Analyzing and transforming raw training data...
Writing transformed training data...
Reading and parsing raw evaluation data...
Transforming raw evaluation data...
Writing transformed evaluation data...
Writing transform artefacts...


RuntimeError: CalledProcessError(1, ['/usr/bin/python3', '-m', 'pip', 'download', '--dest', '/tmp/tmp39twcfx0', 'apache-beam==2.13.0.dev0', '--no-deps', '--no-binary', ':all:'])

### Check the outputs

In [ ]:
!gsutil ls {OUTPUT_DIR}/* 

In [ ]:
!gsutil ls {OUTPUT_DIR}/transform/transform_fn/*

In [ ]:
!gsutil cat output/transform/transformed_metadata/schema.pbtxt

## License

Copyright 2019 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0.

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

---
This is not an official Google product. The sample code provided for educational purposes only.
---